<a href="https://colab.research.google.com/github/lcipolina/escher/blob/master/Img_Class/ClipReranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab version of this repo

https://github.com/mehdidc/clip_rerank

Thanks to @pbaylies and @afiaka87 and @alstroemeria313

In [ ]:
!git clone https://github.com/openai/CLIP && cd CLIP && pip install -e .

Cloning into 'CLIP'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 212 (delta 19), reused 24 (delta 9), pack-reused 168
Receiving objects: 100% (212/212), 11.48 MiB | 9.67 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Obtaining file:///content/CLIP
     |████████████████████████████████| 53 kB 2.3 MB/s 
  Running setup.py develop for clip


In [ ]:
!pip install clize --quiet

     |████████████████████████████████| 71 kB 148 kB/s 
     |████████████████████████████████| 69 kB 10.1 MB/s 


In [ ]:
!pip install gdown

In [ ]:
import os
from clize import run
import torch
from CLIP import clip
from PIL import Image
from glob import glob

In [ ]:
import gdown

# a file
url = "https://drive.google.com/file/d/1BPsYLdfE9TPXTsx05pfS72w0uyzzKoHu/view?usp=sharing"
output = "a.zip"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1BPsYLdfE9TPXTsx05pfS72w0uyzzKoHu
To: /content/a.zip
100%|██████████| 35.3M/35.3M [00:00<00:00, 38.5MB/s]


'a.zip'

In [ ]:
import gdown

# a file
url = "https://drive.google.com/file/d/1cBFlNhbnkLQ0_sOM10OcV1Jtd5wwxUOr/view?usp=sharing"
output = "model.ckpt"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1cBFlNhbnkLQ0_sOM10OcV1Jtd5wwxUOr
To: /content/model.ckpt
100%|██████████| 9.80M/9.80M [00:00<00:00, 104MB/s]


'model.ckpt'

In [ ]:
!unzip a.zip -d a

Archive:  a.zip
   creating: a/0-bis/
  inflating: a/0-bis/.DS_Store       
  inflating: a/__MACOSX/0-bis/._.DS_Store  
   creating: a/0-bis/BAD/
   creating: a/0-bis/GOOD/
  inflating: a/0-bis/BAD/output_13.png  
  inflating: a/__MACOSX/0-bis/BAD/._output_13.png  
  inflating: a/0-bis/BAD/output_2 (3).png  
  inflating: a/__MACOSX/0-bis/BAD/._output_2 (3).png  
  inflating: a/0-bis/BAD/output_15 (1).png  
  inflating: a/__MACOSX/0-bis/BAD/._output_15 (1).png  
  inflating: a/0-bis/BAD/output_6 (1).png  
  inflating: a/__MACOSX/0-bis/BAD/._output_6 (1).png  
  inflating: a/0-bis/BAD/output_220220414-104425.png  
  inflating: a/__MACOSX/0-bis/BAD/._output_220220414-104425.png  
  inflating: a/0-bis/BAD/output_1 (5).png  
  inflating: a/__MACOSX/0-bis/BAD/._output_1 (5).png  
  inflating: a/0-bis/BAD/output_0 (4).png  
  inflating: a/__MACOSX/0-bis/BAD/._output_0 (4).png  
  inflating: a/0-bis/BAD/output_3 (2).png  
  inflating: a/__MACOSX/0-bis/BAD/._output_3 (2).png  
  inflating: a/0-

In [ ]:
from PIL import Image
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F


class Discriminator(nn.Module):
    def __init__(self, in_feature_size = 118, batch_size = 1):
        
        super(Discriminator, self).__init__()
        self.batch_size = batch_size
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.bn1 = nn.BatchNorm2d(64, affine=False)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.bn2 = nn.BatchNorm2d(128, affine=False)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.bn3 = nn.BatchNorm2d(256, affine=False)
        self.conv4 = nn.Conv2d(256, 512, 3)
        self.bn4 = nn.BatchNorm2d(512, affine=False)
        self.conv5 = nn.Conv2d(512, 1, 3)
        self.fc1 = nn.Linear(in_feature_size * in_feature_size , 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 2)
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.conv5(x)
        x = x.view(1, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

device = "cuda"
net = Discriminator()
net = nn.DataParallel(net, device_ids=[0])
net.load_state_dict(torch.load("model.ckpt"))
net.eval()

DataParallel(
  (module): Discriminator(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
    (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (conv5): Conv2d(512, 1, kernel_size=(3, 3), stride=(1, 1))
    (fc1): Linear(in_features=13924, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=8, bias=True)
    (fc3): Linear(in_features=8, out_features=2, bias=True)
  )
)

In [ ]:
transform = transforms.Compose([
            transforms.Resize([128, 128]), # Resizing the image as the VGG only take 224 x 244 as input size
            transforms.RandomHorizontalFlip(), # Flip the data horizontally
            #TODO if it is needed, add the random crop
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5), std=(0.5))
        ])


In [ ]:
def main(pattern, text="a gallery with arches wooden windows and arcades and floors with tiles escher", target_folder="reranked", top:int=None):
    device = "cuda"
    model, preprocess = clip.load("ViT-B/32", device=device)
    paths = glob(pattern, recursive = True)
    score = {}
    score_good = {}
    score_bad = {} 
    for path in paths:
        image = preprocess(Image.open(path)).unsqueeze(0).to(device)
        image_disc =  torch.unsqueeze(transform(Image.open(path)), 0)
        text = "a gallery with arches wooden windows and arcades and floors with tiles escher"
        text = clip.tokenize([text]).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            logits_per_image, logits_pertext = model(image, text)
            logits_disc = torch.nn.functional.softmax(net(image_disc))
            index = torch.argmax(logits_disc).item()
            if index == 0:
              score_bad[path] = logits_per_image.item()
            else:
              score_good[path] = logits_per_image.item()
            score[path] =  logits_per_image.item()
            print(path, score[path])
    paths = sorted(paths, key=lambda f:-score[f])
    if top:
        paths = paths[0:top]
    os.makedirs(target_folder, exist_ok=True)
    for i, path in enumerate(paths):
        out_path = os.path.join(target_folder, f"{i:010d}.png")
        Image.open(path).save(out_path)
    return score, score_bad, score_good
#if name == "main":
#    run(main)

In [ ]:
score, score_bad, score_good = main('./a/**/*.png')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


./a/0-bis/BAD/output_2 copy.png 26.90625
./a/0-bis/BAD/output_19 (1).png 26.875
./a/0-bis/BAD/output_3 (5).png 25.640625
./a/0-bis/BAD/output_7 (5).png 26.421875
./a/0-bis/BAD/output_10 (1).png 27.75
./a/0-bis/BAD/output_16 (2).png 26.828125
./a/0-bis/BAD/output_1720220414-104425.png 26.90625
./a/0-bis/BAD/output_10 copy.png 26.703125
./a/0-bis/BAD/output_17.png 26.46875
./a/0-bis/BAD/output_8 (2).png 26.28125
./a/0-bis/BAD/output_3-20220414-105111.png 27.125
./a/0-bis/BAD/output_920220414-104425.png 26.84375
./a/0-bis/BAD/output_9.png 26.0
./a/0-bis/BAD/output_1 copy.png 27.078125
./a/0-bis/BAD/output_14 (2).png 27.15625
./a/0-bis/BAD/output_8 (4).png 26.0625
./a/0-bis/BAD/output_5 (2).png 26.640625
./a/0-bis/BAD/output_9 (1).png 28.171875
./a/0-bis/BAD/output_6 (2).png 26.59375
./a/0-bis/BAD/output_6 (2) copy.png 27.75
./a/0-bis/BAD/output_9 (5).png 27.40625
./a/0-bis/BAD/output_19 (4).png 26.8125
./a/0-bis/BAD/output_8 (1).png 27.3125
./a/0-bis/BAD/output_6 (5).png 27.6875
./a/0-bis

In [ ]:
import numpy as np
from scipy import stats

def find_stats(score, title):
  scores_raw = np.array(list(score.values()))
  print(title)
  print(f'Mean: {np.mean(scores_raw)}')
  print(f'Median: {np.median(scores_raw)}')
  print(f'Variance: {np.std(scores_raw)}')


find_stats(score, "Total Stats")
find_stats(score_good, "Stats Good Images")
find_stats(score_bad, "Stats bad images")

Total Stats
Mean: 26.594597868217054
Median: 26.6015625
Variance: 0.8386852956045462
Stats Good Images
Mean: 26.557291666666668
Median: 26.46875
Variance: 0.8221528202570558
Stats bad images
Mean: 26.615625
Median: 26.625
Variance: 0.8471379092630704
